In [1]:
import scipy as sp
import numpy as np
import meshplot as mp
import time 
import torch

from meshplot import plot, subplot, interact
from meshplot import Viewer

import sys
src_path = '../'
sys.path.append(src_path)

from smpl_torch_batch import SMPLModel


## PLOT JIGGLING SMPL ANIMATION

In [2]:

with np.load("../results/optim_parameters_smpl_budget_1000.npz") as data:
    betas_optim = data['arr_0']
    pose_optim = data['arr_1']
    #trans_optim = data['arr_2']
    
print(betas_optim.shape, pose_optim.shape) #, trans_optim.shape)

(268, 10) (268, 72)


In [3]:
device = "cpu"

training_data = torch.load('../data/50004_dataset.pt')
data_loader = torch.utils.data.DataLoader(training_data, batch_size=1, shuffle=False)
smpl_model = SMPLModel(device=device, model_path='../body_models/smpl/female/model.pkl')


In [4]:
for data in data_loader:
    beta_pose_trans_seq = data[0].squeeze().type(torch.float64)
    betas = beta_pose_trans_seq[:,:10]
    pose = beta_pose_trans_seq[:,10:82]
    trans = beta_pose_trans_seq[:,82:] 
    
    target_verts = data[1].squeeze()
        
    smpl_verts, joints, affine = smpl_model(betas, pose, trans, return_T=True)
    smpl_verts_optim, joints_optim, affine_optim = smpl_model(
                                                                torch.Tensor(betas_optim).type(torch.float64), 
                                                                torch.Tensor(pose_optim).type(torch.float64), 
                                                                torch.Tensor(trans).type(torch.float64),
                                                                #torch.Tensor(trans_optim).type(torch.float64),
                                                                return_T = True
                                                               )
    
    f = smpl_model.faces

    break

In [5]:
print(torch.sum((smpl_verts - smpl_verts_optim) ** 2))

tensor(995.7208, dtype=torch.float64)


In [6]:
def view_animation(V : np.ndarray, 
               f : np.ndarray,
                  num_iters=1):
 
    v = V[0]
    p = mp.plot(v, f)

    anim_length = target_verts.shape[0]
    for j in range(num_iters):
        for i in range(anim_length):
            v_new = V[i]
            p.update_object(vertices=v_new)
            v = v_new
            time.sleep(0.1) # depending on how long your simulation step takes you want to wait for a little while

In [7]:
#V = smpl_verts_optim.detach().numpy()
#view_animation(V, f)

## PLOT UNPOSED DFAUST ANIMATION

In [8]:
def unpose_verts_batch(target_verts, trans_vec, T):
    T_inv = torch.inverse(T)
    num_batch = target_verts.shape[0]
    
    tmp_verts = torch.clone(target_verts)
    tmp_verts -= torch.reshape(trans_vec, (num_batch, 1, 3)) 
    
    v_homo = torch.cat([tmp_verts,torch.ones([num_batch, 6890, 1])], dim=2)
    v_unposed = torch.matmul(T_inv, torch.unsqueeze(v_homo, dim=-1)).squeeze()[:, :,:3]
    
    return v_unposed

In [10]:

v_unposed = unpose_verts_batch(target_verts, trans, affine_optim)
V = v_unposed.detach().numpy()
view_animation(V, f)

Renderer(camera=PerspectiveCamera(children=(DirectionalLight(color='white', intensity=0.6, position=(0.0168386…